In [ ]:
import sys
sys.path.append('../../olympus/lib/python3.10/site-packages/')
from ananke.models.collection import Collection
from ananke.configurations.collection import MergeConfiguration
from ananke.schemas.event import RecordType
from ananke.configurations.events import EventRedistributionMode
from ananke.configurations.events import (
    RedistributionConfiguration,
    Interval,
    EventRedistributionMode,
)
from ananke.configurations.collection import HDF5StorageConfiguration
import os
import time
import logging
import ananke.models.collection
import importlib
importlib.reload(ananke.models.collection)
from ananke.models.collection import Collection#, LegacyCollection
from ananke.configurations.collection import HDF5StorageConfiguration
logging.getLogger().setLevel(logging.INFO)

from ananke.configurations.collection import (
    HDF5StorageConfiguration,
    MergeConfiguration, MergeContentConfiguration,
)
from ananke.visualisation.detector import get_detector_scatter3ds
import numpy as np
import matplotlib.pyplot as plt
import time

logging.getLogger().setLevel(logging.INFO)

import cProfile
import pstats
from joblib import Parallel, delayed

In [3]:
config=HDF5StorageConfiguration(data_path='data/HexCascadeBenchMark.h5',read_only=False)
C1=Collection(config)
C1.open()
display(C1.storage.get_records().df.shape)
C1.close()

(10, 13)

In [4]:
config=HDF5StorageConfiguration(data_path='data/1.h5',read_only=False)
C1=Collection(config)

In [5]:
C1.open()
print(type(C1.storage.store))
C1.close()

<class 'pandas.io.pytables.HDFStore'>


In [3]:
path='data/'
configuration = MergeConfiguration.parse_obj(
    {
        'in_collections': [
            {
                'type': 'hdf5',
                'data_path':'data/HexCascadeBenchMark.h5',
                'read_only':'False',
            },
            {
                'type': 'hdf5',
                'data_path': 'data/Hex_electrical_noise_10_BM.h5',
                'read_only':'False',
            },
            {
                'type': 'hdf5',
                'data_path': 'data/HexBioluminescenceBenchMark.h5',
                'read_only':'False',
            },
        ],
        'out_collection': {
                'type': 'hdf5',
                'data_path': 'data/OutCollection.h5',
                'read_only':'False',
        },
        'content': [
            {
                'primary_type': RecordType.CASCADE.value,
                'secondary_types': [RecordType.ELECTRICAL.value,RecordType.BIOLUMINESCENCE],
                'number_of_records': 10,
                'interval': {
                    'start': 0,
                    'end': 1000,
                }
            },
        ],
        'redistribution': {
            'interval': {
                'start': 0,
                'end': 1000
            },
            'mode': EventRedistributionMode.CONTAINS_EVENT
        }
           
    }
)

print(configuration)

in_collections=[HDF5StorageConfiguration(type=<StorageTypes.HDF5: 'hdf5'>, read_only=False, batch_size=100, data_path='data/HexCascadeBenchMark.h5', complevel=3, complib='lzo', optlevel=6), HDF5StorageConfiguration(type=<StorageTypes.HDF5: 'hdf5'>, read_only=False, batch_size=100, data_path='data/Hex_electrical_noise_10_BM.h5', complevel=3, complib='lzo', optlevel=6), HDF5StorageConfiguration(type=<StorageTypes.HDF5: 'hdf5'>, read_only=False, batch_size=100, data_path='data/HexBioluminescenceBenchMark.h5', complevel=3, complib='lzo', optlevel=6)] tmp_collection=HDF5StorageConfiguration(type=<StorageTypes.HDF5: 'hdf5'>, read_only=False, batch_size=100, data_path='/raven/u/arego/olympus/lib/python3.10/site-packages/ananke/configurations/../../_tmp_62b2855d-9d75-42a0-828d-5d6829437440data.h5', complevel=3, complib='lzo', optlevel=6) out_collection=HDF5StorageConfiguration(type=<StorageTypes.HDF5: 'hdf5'>, read_only=False, batch_size=100, data_path='data/OutCollection.h5', complevel=3, com

In [4]:
with cProfile.Profile() as profile2:
    collection=Collection.from_merge(configuration)

INFO:root:Starting to merge collections with config.
INFO:root:Starting to create joined temporary collection.
100it [00:05, 16.73it/s]              
INFO:Collection:Starting to append collection.
100it [00:00, 1164.88it/s]            
INFO:Collection:Finished to append collection.
INFO:Collection:Starting to append collection.
100it [00:00, 574.96it/s]             
INFO:Collection:Finished to append collection.
INFO:root:Finished creating joined temporary collection.
INFO:Collection:Starting to redistribute with mode: contains_event
 33%|███▎      | 10/30 [1:41:38<3:23:16, 609.84s/it]
INFO:Collection:Finished to redistribute with mode: contains_event
INFO:root:Starting to create 10 RecordType.CASCADE records
INFO:root:Secondary types: 20, 21
100%|██████████| 10/10 [01:01<00:00,  6.17s/it]
INFO:root:Finished to create 10 RecordType.CASCADE records
INFO:root:Finished to merge collections with config.


In [ ]:
path='data/'
configuration = MergeConfiguration.parse_obj(
    {
        'in_collections': [
            {
                'type': 'hdf5',
                'data_path':'data/HexCascadeBenchMark.h5',
                'read_only':'False',
            },
            {
                'type': 'hdf5',
                'data_path': 'data/Hex_electrical_noise_10_BM.h5',
                'read_only':'False',
            },
            {
                'type': 'hdf5',
                'data_path': 'data/HexBioluminescenceBenchMark.h5',
                'read_only':'False',
            },
        ],
        'out_collection': {
                'type': 'hdf5',
                'data_path': 'data/4.h5',
                'read_only':'False',
        },
        'content': [
            {
                'primary_type': RecordType.CASCADE.value,
                'secondary_types': [RecordType.ELECTRICAL.value,RecordType.BIOLUMINESCENCE],
                'number_of_records': 10,
                'interval': {
                    'start': 0,
                    'end': 1000,
                }
            },
        ],
        
           
    }
)

print(configuration)

In [ ]:
with cProfile.Profile() as profile2:
    collection=Collection.from_merge(configuration)

In [12]:
results1.print_stats('site-packages/olympus')

         436026673 function calls (435720231 primitive calls) in 6167.725 seconds



In [8]:
results1 = pstats.Stats(profile2)
results1.sort_stats(pstats.SortKey.CUMULATIVE)
results1.print_stats('ananke')

         436026673 function calls (435720231 primitive calls) in 6167.725 seconds

   Ordered by: cumulative time
   List reduced from 2147 to 55 due to restriction <'ananke'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001 6167.728 6167.728 /raven/u/arego/project/Experimenting/../../olympus/lib/python3.10/site-packages/ananke/models/collection.py:390(from_merge)
        1    0.001    0.001 6098.463 6098.463 /raven/u/arego/project/Experimenting/../../olympus/lib/python3.10/site-packages/ananke/models/collection.py:156(redistribute)
       20    0.001    0.000 6057.184  302.859 /raven/u/arego/project/Experimenting/../../olympus/lib/python3.10/site-packages/ananke/services/collection/storage.py:751(__del_data)
       10    0.000    0.000 6057.071  605.707 /raven/u/arego/project/Experimenting/../../olympus/lib/python3.10/site-packages/ananke/services/collection/storage.py:593(del_hits)
      145    0.010    0.000  101.020    0.697 /rave

In [ ]:
results1.dump_stats('reorg1.prof')

In [ ]:
result=pstats.Stats('reorg.prof')

In [ ]:
result.sort_stats(pstats.SortKey.CUMULATIVE)
result.print_stats('ananke')

In [ ]:
config=HDF5StorageConfiguration(data_path='data/HexCascadeBenchMark.h5',read_only=False)
C1=Collection(config)
C1.open()
hits=C1.storage.get_hits()
sources=C1.storage.get_sources()
records=C1.storage.get_records()
detector=C1.storage.get_detector()
C1.close()

In [ ]:
for i in records.df.itertuples():
    print(i)
    

In [ ]:
from joblib import Parallel, delayed

def print_stats(rec):
    i=getattr(rec, "record_id")
    return hits.get_by_record_ids(i).get_statistics()
def nesting():
    x=np.zeros((3,3))
    for i in range(10):
        x+=np.ones((3,3))*i
        y=Parallel(n_jobs=8)(delayed(print_stats)(rec) for rec in records.df.itertuples())
    return y

with cProfile.Profile() as profile:
    x=Parallel(n_jobs=8)(delayed(print_stats)(rec) for rec in records.df.itertuples())
x

In [ ]:
for i in records.df.itertuples():
    print(getattr(i, "record_id"),type(i))

In [ ]:
for i in records.itertuples():
    print(getattr(i, "record_id"))

In [ ]:
C1.open()

In [ ]:
configuration.redistribution

In [ ]:
C1.close()

In [ ]:
records.df

In [ ]:
C1.storage.get_records().df

In [ ]:
C1.close()

In [ ]:
for i in range(10):
    """hi"""
    print(i)

In [ ]:
Parallel(n_jobs=8)(
            delayed()(record)#, rng, redistribution_configuration, record_types)
            for record in records.df.itertuples()
        )

In [ ]:
records.record_ids

In [ ]:
from joblib import Parallel, delayed

class ParallelProcessor: # Number of jobs for parallel processing

    def square(self, x):
        """Calculates the square of a number."""
        return x ** 3

    def process_numbers(self, numbers):
        """Processes a list of numbers in parallel."""
        print("Processing numbers in parallel...")
        results = Parallel(n_jobs=8)(
            delayed(self.square)(num) for num in numbers
        )
        return results

# Example usage
if __name__ == "__main__":
    processor = ParallelProcessor()
    numbers = [1, 2, 3, 4, 5]
    squared_numbers = processor.process_numbers(numbers)
    print("Squared numbers:", squared_numbers)


In [ ]:
import testingimport as ti

In [ ]:
processor = ti.ParallelProcessor()
numbers = [1, 2, 3, 4, 5]
squared_numbers = processor.process_numbers(numbers)
print("Squared numbers:", squared_numbers)

In [ ]:
for i,j in enumerate(range(5,10)):
    print(i,j)

In [5]:
C1.open()

In [6]:
C1.storage.get_records().df

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,energy,length,time,type,particle_id,duration
0,-51.148253,140.009025,378.404412,0.815855,0.475162,0.329547,9,55427.232682,3000.0,0.0,1,11,NaN
1,105.854169,-126.931308,288.042876,-0.530766,0.818127,-0.221258,4,2305.681058,3000.0,0.0,1,11,NaN
2,20.848573,172.442361,250.813873,-0.149179,0.229946,-0.961702,7,116.686889,3000.0,0.0,1,11,NaN
3,38.257116,-41.263073,306.950846,-0.528620,0.814819,-0.237973,6,2155.494073,3000.0,0.0,1,11,NaN
4,105.854169,-126.931308,288.042876,-0.530766,0.818127,-0.221258,10,2305.681058,3000.0,0.0,1,11,NaN
5,-154.788002,-33.427564,443.736516,-0.490772,0.756479,0.432298,2,32101.272279,3000.0,0.0,1,11,NaN
6,170.397953,66.320281,-68.194255,0.711295,0.414265,0.567842,8,1568.839788,3000.0,0.0,1,11,NaN
7,38.257116,-41.263073,306.950846,-0.528620,0.814819,-0.237973,11,2155.494073,3000.0,0.0,1,11,NaN
8,84.605689,103.181461,-11.308332,0.560505,0.326443,-0.761097,12,34240.241604,3000.0,0.0,1,11,NaN
9,-54.260814,20.774832,275.013752,-0.542991,0.587868,0.599643,13,261.866222,3000.0,0.0,1,11,NaN


In [19]:
C1.storage.get_sources().df

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,time,number_of_photons,type
0,105.676254,-126.896660,288.141296,-0.530766,0.818127,-0.221258,4,0.336900,334,0
1,105.540062,-126.767609,288.041290,-0.530766,0.818127,-0.221258,4,1.004028,28885,0
2,105.502151,-126.546753,288.044952,-0.530766,0.818127,-0.221258,4,1.671156,324677,0
3,105.404381,-126.443367,287.757812,-0.530766,0.818127,-0.221258,4,2.338284,1554260,0
4,105.438347,-126.196587,287.684937,-0.530766,0.818127,-0.221258,4,3.005413,4698470,0
...,...,...,...,...,...,...,...,...,...,...
7559,17.364393,178.083145,227.850250,-0.149179,0.229946,-0.961702,13,79.725151,0,0
7560,17.450766,178.046066,227.620010,-0.149179,0.229946,-0.961702,13,80.392281,0,0
7561,17.363331,178.119949,227.443283,-0.149179,0.229946,-0.961702,13,81.059402,0,0
7562,17.259926,177.910629,227.201309,-0.149179,0.229946,-0.961702,13,81.726532,0,0


In [21]:
C1.storage.store.remove(where='((record_id=9))',key='records')

1

In [3]:
from ananke.configurations.collection import HDF5StorageConfiguration
config=HDF5StorageConfiguration(data_path='data/1.h5',read_only=False)
C1=Collection(config)


In [15]:
keys=['records','hits','sources']
C1.open()
display(C1.storage.get_records().df) #to see the data set before deletion
#C1.storage.store.remove(where='((record_id=9))',key=Keys[1])
display(C1.storage.get_hits().df) #to see it after deletion
C1.close()

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,energy,length,time,type,particle_id,duration
0,105.854169,-126.931308,288.042876,-0.530766,0.818127,-0.221258,4,2305.681058,3000.0,0.0,1,11,NaN
1,20.848573,172.442361,250.813873,-0.149179,0.229946,-0.961702,7,116.686889,3000.0,0.0,1,11,NaN
2,38.257116,-41.263073,306.950846,-0.528620,0.814819,-0.237973,6,2155.494073,3000.0,0.0,1,11,NaN
3,105.854169,-126.931308,288.042876,-0.530766,0.818127,-0.221258,10,2305.681058,3000.0,0.0,1,11,NaN
4,170.397953,66.320281,-68.194255,0.711295,0.414265,0.567842,8,1568.839788,3000.0,0.0,1,11,NaN
5,38.257116,-41.263073,306.950846,-0.528620,0.814819,-0.237973,11,2155.494073,3000.0,0.0,1,11,NaN
6,84.605689,103.181461,-11.308332,0.560505,0.326443,-0.761097,12,34240.241604,3000.0,0.0,1,11,NaN
7,-54.260814,20.774832,275.013752,-0.542991,0.587868,0.599643,13,261.866222,3000.0,0.0,1,11,NaN


,time,string_id,module_id,pmt_id,record_id,type
0,887.258960,1,5,15,4,1
1,892.335039,1,5,15,4,1
2,886.597592,1,5,15,4,1
3,886.999259,1,5,15,4,1
4,892.332709,1,5,15,4,1
...,...,...,...,...,...,...
3054962,808.000000,13,19,6,13,21
3054963,612.000000,13,19,7,13,21
3054964,622.000000,13,19,10,13,21
3054965,690.500000,13,19,3,13,21


In [16]:
C1.open()
storage=C1.storage.store
C1.close()

In [19]:
C1.open()
storage.remove(where='((record_id=4))',key='hits')

86177